# M5 Forecasting - Accuracy: Review
***

Finally got around to seeing through my second Kaggle competition to the end - lucky I picked a pretty fun one to commit to! Though I'm sure they're all fun, just wish I had enough time to do them all :)

In this competition, the challenge was to forecast daily individual item sales over 28 days across 10 Walmart stores - all in all, around 32'000 different products! I wanted to be a bit more organised in my approach this time compared to a year ago when I did my first competition, so I set up a bit of a 3-stage pipeline:
1. Feature generation
2. Model training and forecasting for individual items 
3. Forecast compiler to create the submission file

***

## 1. Feature generation script [|link|](https://www.kaggle.com/slashie/m5-feat-gen)

### Overview
This script draws on a bunch of helper functions that I wrote [here](https://www.kaggle.com/slashie/m5-helper) and it's really just trying to do the following:
* Make sense of the price data 
* Aggregate the sales data (I thought it would be useful, but didn't end up using it - see my note below on survivorship bias in the data)
* *Ignore the calendar data completely... yeah, I left that to the training stage, oops...*

In this notebook, the `m5-feat-gen` input folder contains all the datasets that I produced and used in the model training stage. 

### Price-based features
Overall, the key aspects of my approach were:
* **Price indexation**: in each week, I normalized an item's price by its median price for the sample, to get an idea of whether it's cheaper or more expensive than usual. Also, this could potentially help with making its price comparable to other items or aggregated categories.
* **Week-on-week price changes**: I also looked at simple percent changes in an item's price (is it going on discount this week?), as well as its lag (was it on discount last week or the week before etc.?)
* **Price aggregation**: I also thought that if I aggregated prices by department, category, store or state, I could create relative pricing features that would say how the item is being priced in comparison to other items overall, which might be useful in predicting sales. I even created my own price-based groups to cluster items with similar prices in each category - in `price_grps.csv`, you can see my handywork in the ridiculous ID names e.g. `FOODS_1AAABBBACCCCCCCCCC` :)

### Survivorship bias in the data

While I was having the time of my life crunching the price features, I failed to realize a major flaw in the dataset. The only items that are visible in the training data are those that are still selling during the prediction period! *Therefore, any items that were discontinued during the training period cannot be accounted for - hence the survivorship bias.* This is a major issue, because any aggregate features are likely missing a large chunk of the entire picture, so all the effort I invested in creating all those aggregate features was probably not the best use of my time. 

***

## 2. Sample training/forecasting script [|link|](https://www.kaggle.com/slashie/m5-forecast-ca1)

This is 1 of 10 scripts that I ran for each of the 10 stores - since I was running short on time with the competition deadline, I decided to maximize my usage of all 10 CPUs I have on Kaggle by running all 10 scripts in parallel :)

Most of the magic here is created behind the scenes by the helper functions in [this utility script](https://www.kaggle.com/slashie/m5-helper-s2). I will demostrate most of this in this notebook, so won't say much more here.

***

## 3. Submission file compiler [|link|](https://www.kaggle.com/slashie/m5-forecast-all)

Finally, the one script to rule them all compiles the predictions from the 10 parallel scripts - I think this was my best submission, but not the one I chose for final scoring, though it wouldn't have made a huge difference, I'd still been in the top 8% with my Bronze medal...

***
<br>

# M5 Forecasting - Accuracy: Demo

This brings me to this quick demo which is quite similar to the notebook I used to train and test various model ideas throughout the competition...


## Packages

Nothing unexpected here - but I do have quick fun note about why I am using `colorama`: 

Basically, Kaggle decided to update something in the last few days of the competition that messed up the font colour of `print()` output for dark-mode i.e. it was black-on-black and couldn't see a thing. It seems to have been fixed now, but at the time I was too stubborn to change to light-mode and instead used `colorama` to control the colour of `print()` outputs - not sure if the effort paid for itself given how precious time gets close to the deadline...

In [ ]:
# The standard stuff
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Me being OCD about using dark-mode on Kaggle
from colorama import Fore

# My helper utility script
from m5_helper_s2 import load_stage2_data, gen_all_cal_feat, gen_item_df, train_lgbm

***

## Setup

### Load training data

All the relevant competition data files, as well as my `m5-feat-gen` output:

In [ ]:
files = load_stage2_data()
print(Fore.CYAN+str(files.keys()))

### Calendar features

Given the price-based and aggregate price/sales features turned out to be a bit of a rabbit hole, I really should have spent more time on time-based features. Nevertheless, here's what I did with the *time* (oh snap!) I had:
* **Sine/cosine features**: these are an awesome way to capture the cyclical/seasonal nature of time - particularly when looking over an entire year. The `year_sin` and `year_cos` features were always super-important in my item-level LGBoost models. Read [this helpful and concise blogpost](https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/) for more info.
* **Special days**: I figured/hoped the sine/cosine features above would be able to capture the holidays that always happen on the same date each year, so I only focused on a subset of special days that occur on different dates each year. Instead of one-hot-encoding, I built the features the following way:
  * For `pre_` features, the value is 1 on the event day and builds up to it from 0 of over N days prior
  * For `post_` features, the value is 1 on the event day and decreases to 0 of over N days after
  * The idea is that item sales likely starting picking up before the event (e.g. Easter chocolate) and then there may be sales after (e.g. Easter chocolate :))
* **Trend variables**: I am not a fan of trend variables in tree-based models (I think they can cause models to overfit and render them useless for prediction i.e. by splitting away too little training data for latter periods), but I thought having a month-level trend (`ymidx`) would be broad enough to capture any structural breaks in item sales e.g. if an item becomes popular (or otherwise) all of a sudden. 
* [**SNAP**](https://en.wikipedia.org/wiki/Supplemental_Nutrition_Assistance_Program): definetely worth including

<br>

Creating calendar feature data using the `gen_all_cal_feat()` function:

In [ ]:
cal_df = gen_all_cal_feat(files["calendar"])
cal_df.sample(3)

Just a quick graph to show what my *sine/cosine* and *special-day* features look like:

In [ ]:
fig, ax = plt.subplots(2, 1, facecolor='w', figsize=(16,6))
cal_df[['year_sin', 'year_cos']].plot(ax=ax[0])
ax[0].set_title("Sine and cosine features")
cal_df[['pre_Easter', 'post_Easter']].plot(ax=ax[1])
ax[1].set_title("Example of special-day encoding")
fig.tight_layout(pad=3.0)
plt.show()

***
## Model training

### Single item: training data assembly

Since I chose to model each item's sales individually, I wrote the `gen_item_df()` function to compile the following features for each item:
* **Calendar features**: as described in the section above
  * Note I only keep the `snap_{state}` feature that corresponds to the item's state
* **Price features**: extracted from the `m5-feat-gen` output files:
  * `"item_prices.csv"`: price features for the item: *price index, percent change, lagged percent change*
  * `"grp_prices.csv"`: same price features, but for groups relevant to the item (e.g. the item's department, category, store or state - including combinations of)
  * `"price_grps.csv"`: confusing naming, but this is what maps the item's ID to the price-based clusters that I generated
* **Lagged sale features**: I didn't think daily lags would be much use, so, to capture auto-regressive behaviour, I used a weekly lag for the following:
  * `sales_lag_1`: number of sales on the same day last week 
  * `mean_sales_lag_1`: mean number of sales last week
  * `std_sales_lag_1`: standard deviation of sales last week
  * `n0_sales_lag_1`: number of days with 0-sales last week
  * `max_sales_lag_1`: maximum number of sales last week
  
I also do a bit of cleaning:
* Filtering out days where item is not available: *the training data records 0 sales for days that should be N/A based on the lack of price data i.e. these are products that are being introducing after the start of the training period*
* Dropping columns with 0-variance: *this most commonly happens when an item's price never changes during the training sample*
* Dropping highly-correlated columns: *some price features can be very similar*

In [ ]:
item_id = np.random.choice(files["price_grps"]["id"])
item_df = gen_item_df(files, item_id, cal_df, verbose=True, pca_min_var=None)
print(Fore.CYAN+"%s has %d rows and %d features"%(item_id, item_df.shape[0], item_df.shape[1]))
print(Fore.CYAN+"Sales have a mean of %.2f and st. dev. of %.2f"%(item_df.TARGET.mean(), item_df.TARGET.std()))

### Single item: training & forecasting

Admittedly, I did not get enough time to explore a multitude of models/algorithms, but the main ones I tested were XGBoost and LightGBM. In the end, I stuck to LightGBM, because it appeared to be faster and I was about to train ~32'000 separate models! I did some basic tuning of the hyper-parameters that would apply to *all items* (individually tuning parameters would have taken a long time... there may have been a more methodical solution here, but I had to wing it quickly to get my submission out in time):
* Relatively low learning rate `eta` with variable number of boosting rounds appeared to work well (i.e. I would at least allow the number of boosting rounds to be determined at the item level)
* Bagging, particularly for features (note that `feature_fraction = 0.55`, whereas `bagging_fraction = 0.9), seemed to be quite useful
* Because some outlier events can be rare but informative/non-spurious, I allowed the model to have a lower `min_data_in_leaf` threshold i.e. there can be leaves with relatively few data points (also there is not a lot of data - typically 1900 days per item)

My `train_lgbm()` function implements the following approach to train and predict sales for each item:
1. Split apart training (days 1 to 1912), validation (includes days 1913 to 1940) and evaluation data (only days 1941 to 1968 for final set of predictions)
2. Finalise training features
  * I added a `final90days` binary feature to capture any structural shifts in sales dynamics close to the prediction period
  * I dropped daily (`idx`) and weekly (`wk_idx`) trend variables to avoid overfitting
3. Train the LightGBM model on the training data and use the validation data, coupled with an `early_stopping_rounds` of 30, to determine the optimal number of boosting rounds (capped at 150)
4. Train the LightGBM model on the training plus validation data, using the number of boosting rounds from step 3, and generate predictions for the evaluation period i.e. for the submission file

Note that, since I use weekly lags, I had to write a special prediction function called `staggered_pred()`, which:
* Makes predictions for one week at the time
* Uses the predictions from the previous week to create the lag features needed to predict the current week's sales
* I did this in order to avoid training different models for different horizons - which may have been more accurate, but would have required a lot of computational time
* For the validation data, I can actually check if there's a major difference between using predicted values versus actual values to create the lag features - doesn't seem like much, but I guess it varies by the extent to which the lag features are important predictors for an item's sales

Here, I test out the `train_lgbm()` function on the randomly-selected item from above, enabling the `show_plots` option to see:
* How predicted sales compare to actual sales
* The feature importance rankings

In [ ]:
params = {
    "objective": "rmse",
    "boosting": "gbdt",
    "eta": 0.03,
    "bagging_freq": 1,
    "bagging_fraction": 0.9,
    "feature_fraction": 0.55,
    "min_data_in_leaf": 5
}
final_model = train_lgbm(item_df, params, X_drop=['idx', 'wk_idx'],
                         show_plots=True, return_model=True)

### Multiple Items

Finally, I wanted to get a good idea of how my model could generalize across a randomly selected sample of `n` items:
* Keeping track of the `RMSE` as well as the competition's `RMSSE` measure
* Keeping track of the time taken so as to get an idea of total computation time when scaling to cover all items

In [ ]:
n = 100
keep_score = {"id":[], "RMSSE":[], "RMSE":[], "RMSE_train":[], "Num_Rounds":[]}
item_list = files["price_grps"]["id"].sample(n).values
item_list_val = [i.replace("evaluation","validation") for i in item_list]
this_sub = files["sample_submission"].copy().query("id in @item_list | id in @item_list_val")

start = datetime.now()
for item_id, item_id_val in zip(item_list, item_list_val):
    item_df = gen_item_df(files, item_id, cal_df, verbose=False)
    y_valid_pred, y_eval_pred = train_lgbm(item_df, params, keep_score=keep_score, X_drop=['idx', 'wk_idx'])
    this_sub.loc[this_sub['id']==item_id_val, "F1":"F28"] = y_valid_pred
    this_sub.loc[this_sub['id']==item_id, "F1":"F28"] = y_eval_pred
    keep_score["id"].append(item_id)
print(Fore.CYAN+'Time taken: %s'%str(datetime.now()-start)[:-7]) 

scores = pd.DataFrame(keep_score)
scores.describe()

***
## With some more time...

You may see at the end of [my helper script](https://www.kaggle.com/slashie/m5-helper-s2) for the model training and forecasting there is a `train_lgbm0()` function. The idea here was to:
* First predict the probability of there being any sales on any given day
* Use this probability as an additional feature in predicting the number of sales

Some of my rough work indicated this would be an effective way to improve `RMSSE` - just wish this idea had not come to me with 1 hour to go to the submission deadline :(